In [12]:
import pandas as pd

In [13]:
split = "eval"
print(split)

eval


In [17]:
main_df = pd.read_csv(f"basic_model/{split}.csv")
print(main_df.shape)
main_df.rename({'predictions': 'main_predictions'}, inplace=True, axis = 1)
main_df.head(3)

(300, 3)


,main_predictions,true_labels,masked_articles
0,fallacy of logic,fallacy of logic,""" Just like students are given a couple of MSK..."
1,appeal to emotion,appeal to emotion,You do n’t have to do this . MSK0 grandmother ...
2,faulty generalization,faulty generalization,"I know MSK0 . MSK0 are all MSK1 . Therefore , ..."


In [20]:
cbr_df = pd.read_csv(f"cbr_simcse_cases/{split}.csv")
print(cbr_df.shape)
cbr_df.rename({'predictions': 'cbr_predictions'}, inplace = True, axis = 1)
cbr_df.head(3)

(300, 4)


,cbr_predictions,true_labels,cbr,masked_articles
0,fallacy of logic,fallacy of logic,MSK0 refer to MSK1 MSK3 all the time when MSK0...,""" Just like students are given a couple of MSK..."
1,appeal to emotion,appeal to emotion,", MSK0 relief MSK1 of MSK0 cut , or death MSK0...",You do n’t have to do this . MSK0 grandmother ...
2,faulty generalization,faulty generalization,"The MSK0 that bit was MSK1 , therefore all MSK...","I know MSK0 . MSK0 are all MSK1 . Therefore , ..."


In [21]:

filtered_cbr_df = pd.read_csv(f"cbr_filtered/{split}.csv")
print(filtered_cbr_df.shape)
filtered_cbr_df.rename({'predictions': 'cbr_filtered_predictions', 'cbr': 'cbr_filtered'}, inplace=True, axis = 1)
filtered_cbr_df.head(3)

(300, 4)


,cbr_filtered_predictions,true_labels,cbr_filtered,masked_articles
0,fallacy of logic,fallacy of logic,MSK0 refer to MSK1 MSK3 all the time when MSK0...,""" Just like students are given a couple of MSK..."
1,appeal to emotion,appeal to emotion,", MSK0 relief MSK1 of MSK0 cut , or death MSK0...",You do n’t have to do this . MSK0 grandmother ...
2,faulty generalization,faulty generalization,"The MSK0 that bit was MSK1 , therefore all MSK...","I know MSK0 . MSK0 are all MSK1 . Therefore , ..."


In [22]:
main_df.drop(columns=['masked_articles'], inplace=True, axis = 1)
cbr_df.drop(columns=['masked_articles', 'true_labels'], inplace=True, axis = 1)
filtered_cbr_df.drop(columns=['masked_articles', 'true_labels'], inplace=True, axis = 1)

In [24]:
all_df = pd.concat([main_df, cbr_df, filtered_cbr_df], axis = 1)

In [28]:
all_df.head(3)

,main_predictions,true_labels,cbr_predictions,cbr,cbr_filtered_predictions,cbr_filtered
0,fallacy of logic,fallacy of logic,fallacy of logic,MSK0 refer to MSK1 MSK3 all the time when MSK0...,fallacy of logic,MSK0 refer to MSK1 MSK3 all the time when MSK0...
1,appeal to emotion,appeal to emotion,appeal to emotion,", MSK0 relief MSK1 of MSK0 cut , or death MSK0...",appeal to emotion,", MSK0 relief MSK1 of MSK0 cut , or death MSK0..."
2,faulty generalization,faulty generalization,faulty generalization,"The MSK0 that bit was MSK1 , therefore all MSK...",faulty generalization,"The MSK0 that bit was MSK1 , therefore all MSK..."


In [29]:
bad_cases = set(all_df[(all_df['main_predictions'] == all_df['true_labels']) & (all_df['cbr_predictions'] != all_df['true_labels'])]['cbr'].tolist())
good_cases = set(all_df[(all_df['main_predictions'] != all_df['true_labels']) & (all_df['cbr_predictions'] == all_df['true_labels'])]['cbr'].tolist())
non_effective_cases = set(all_df[(all_df['main_predictions'] != all_df['true_labels']) & (all_df['cbr_predictions'] != all_df['true_labels'])]['cbr'].tolist())
okay_cases = set(all_df[(all_df['main_predictions'] == all_df['true_labels']) & (all_df['cbr_predictions'] == all_df['true_labels'])]['cbr'].tolist())

In [30]:
all_cbr_filtered = set(all_df['cbr_filtered'].tolist())
len(bad_cases), len(bad_cases - all_cbr_filtered)


(19, 19)

In [36]:
reduced_df = all_df[all_df['cbr'] != all_df['cbr_filtered']]
print(len(reduced_df))
reduced_df_effective = reduced_df[reduced_df['true_labels'] == reduced_df['cbr_filtered_predictions']]
print(len(reduced_df_effective))
reduced_df_effective

21
6


,main_predictions,true_labels,cbr_predictions,cbr,cbr_filtered_predictions,cbr_filtered
28,circular reasoning,circular reasoning,fallacy of extension,Consider the following exchange : Omar : I MSK...,circular reasoning,NaN
57,faulty generalization,faulty generalization,ad hominem,""" MSK0 must be wrong on the issue of taxes bec...",faulty generalization,NaN
108,faulty generalization,faulty generalization,ad populum,"Millions of people are MSK0 , so MSK0 economic...",faulty generalization,NaN
134,circular reasoning,circular reasoning,fallacy of extension,A : I do n't want to visit McDonald 's today ....,circular reasoning,NaN
175,circular reasoning,circular reasoning,false causality,MSK0 are MSK1 . MSK0 can MSK2 . MSK3 are also ...,circular reasoning,NaN
272,circular reasoning,circular reasoning,faulty generalization,"If MSK0 are not with MSK1 , MSK0 are against M...",circular reasoning,NaN


In [37]:
all_df[(all_df['cbr_predictions'] == all_df['true_labels']) & (all_df['cbr_filtered_predictions'] != all_df['true_labels'])]

,main_predictions,true_labels,cbr_predictions,cbr,cbr_filtered_predictions,cbr_filtered
3,circular reasoning,circular reasoning,circular reasoning,""" ' What about MSK1 , ma'am ? '' "" ' I 'm not ...",intentional,""" ' What about MSK1 , ma'am ? '' "" ' I 'm not ..."
52,appeal to emotion,appeal to emotion,appeal to emotion,If MSK0 do n't get MSK1 done then MSK0 will fa...,faulty generalization,If MSK0 do n't get MSK1 done then MSK0 will fa...
85,ad populum,ad populum,ad populum,"MSK0 is a MSK1 organization , so all of MSK0 e...",faulty generalization,"MSK0 is a MSK1 organization , so all of MSK0 e..."
107,false causality,faulty generalization,faulty generalization,MSK0 wants MSK0 to break into the spooky cabin...,false causality,MSK0 wants MSK0 to break into the spooky cabin...
136,fallacy of credibility,fallacy of credibility,fallacy of credibility,"Your honor , if we let Pinocchio get away with...",appeal to emotion,"Your honor , if we let Pinocchio get away with..."
143,ad hominem,fallacy of relevance,fallacy of relevance,MSK0 need to stop allowing MSK1 to increase tu...,ad hominem,MSK0 need to stop allowing MSK1 to increase tu...
156,ad populum,ad populum,ad populum,"MSK0 asked MSK1 about MSK0 , but MSK1 responds...",false causality,"MSK0 asked MSK1 about MSK0 , but MSK1 responds..."
198,appeal to emotion,appeal to emotion,appeal to emotion,MSK0 wakes up one morning to find that MSK0 ha...,faulty generalization,MSK0 wakes up one morning to find that MSK0 ha...
222,fallacy of logic,faulty generalization,faulty generalization,"All MSK0 MSK2 and MSK0 are MSK3 ; therefore , ...",fallacy of logic,"All MSK0 MSK2 and MSK0 are MSK3 ; therefore , ..."
230,ad populum,faulty generalization,faulty generalization,MSK0 is the MSK1 secretary . MSK0 says that MS...,ad populum,MSK0 is the MSK1 secretary . MSK0 says that MS...
